In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split, \
    StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Komoran

# 자연어 처리 순서
1. 데이터의 로드 
2. 데이터 튜닝
3. 데이터 분할
4. 토큰화
5. 백터화 
6. 모델 학습 
7. 평가 

In [ ]:
# 데이터를 로드 
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.head()

In [4]:
# 필요 없는 컬럼을 제외 -> id 제외-> id에 중복 값이 존재하지 않는다. 
df.drop('id', axis=1, inplace=True)

In [ ]:
# 결측치가 존재하는가? 
df.info()

In [6]:
# 결측치가 150000개의 데이터중 5개의 결측치가 관찰 
# 5개니까 굉장히 적은 양 -> 제외
df.dropna(inplace=True)

In [ ]:
# 리뷰 데이터중 중복된 문장이 존재하는가? -> 확인?? -> value_counts()확인
df['document'].value_counts()

In [8]:
# 중복으로 만들어져있는 리뷰 문자들은 제외 -> 과적합 방지 
df.drop_duplicates('document', inplace=True)

In [ ]:
df.info()

In [10]:
# 학습 데이터와 검증 데이터로 데이터를 분할 
X = df['document'].values
Y = df['label'].values

# train test 셋으로 분할 (분류 데이터 -> label의 비율을 유지)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=42
)

In [11]:
# X의 데이터가 문자임으로 토큰화 작업 
komoran = Komoran()
# 사용할 품사 선택 
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'SL', 'MAG']
# 사용하지 않을 단어를 선택 
stop_word = ['하다', '되다']
# 글자 수 제한 
len_word = 2
# 토큰화 함수 정의 
def tokenize(text):
    # 결과를 리스트의 되돌려주기위해 빈 리스트를 생성 
    tokens = []
    for word, pos in komoran.pos(text):
        # 조건1 : 품사에 포함되어있다면
        # 조건2 : 금지어에 포함되어있지 않다면
        # 조건3 : 문자의 길이가 len_word보다 크거나 같다면
        if pos in allow_pos and \
            word not in stop_word and \
            len(word) >= len_word:
            # 3개의 조건을 모두 만족하는 단어를 tokens에 추가 
            tokens.append(word)
    return tokens

In [ ]:
# 백터화 객체 생성 (단어의 중요도를 판단하는 백터화 class 로드)
# 백터화(자연어 데이터에서 사용하는 스케일링 비슷한 작업)
vectorizer = TfidfVectorizer(
    tokenizer=tokenize, 
    ngram_range= (1, 1), 
    min_df= 3, # 3회 이상 나온 단어들을 기준으로 중요도 판단
    lowercase= False
)

In [ ]:
# train 데이터를 이용하여 fit_transform()
# test 데이터는 transform() --> 데이터의 누수 방지 

In [19]:
X_train_vec = vectorizer.fit_transform(X_train)
print(len(vectorizer.get_feature_names_out()))

c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


13575


In [20]:
X_test_vec = vectorizer.transform(X_test)
print(len(vectorizer.get_feature_names_out()))

13575


- train, test 에 모두 fit을 하게 되면 두개의 데이터에서 단어의 추출이 다른 값들을 보인다(데이터의 누수)
- train을 이용하여 fit을 하고 test 데이터는 transform 작업

In [22]:
X_train_vec.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(116945, 13575))

In [23]:
# X_train_vec와 Y_train 데이터를 이용하여 모델에 학습 
model = LinearSVC(C = 1.0)

In [24]:
# 모델에 학습 -> 13575개의 컬럼에서 
# label 0, 1 사이의 규칙을 찾아내는 과정
model.fit(X_train_vec, Y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,None
